In [62]:
!pip install torchaudio -f https://download.pytorch.org/whl/torch_stable.html
!pip install allennlp
!pip install phonemizer
!apt-get install festival espeak-ng mbrola

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://files.pythonhosted.org/packages/e1/ec/eb95ec96dc5374ba339d47c5252d62e9a7b1fbd5c9e4eefebc0d7e25381d/phonemizer-2.2.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5b/a0/0c3fe64787745c39eb3f2f5f5f9ed8d008d9ef22e9d7f9f52f71ea4712f7/segments-2.1.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3f/e6/4faefc40bf1a3e8418cf97b5c3e7619d916692dd4202a5a1e40f36802f78/csvw-1.8.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4e/dd/6afa78497c503e8ad02596b27932df7f3216c0d10b4b7785d942a59b9cfd/clldutils-3.5.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9b/9f/b36f7774ff5ea8e428fdcfc4bb332c39ee5b9362ddd3d40d9516a55221b2/isodate-0.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/78/be/7b8b99fd74ff5684225f50dd0e865393d2265656ef3b4ba9eaaaffe622b8/rfc3986-1.4.0-py2.py3-non

In [63]:
import numpy as np
import os
from torchaudio.datasets.utils import download_url, extract_archive, unicode_csv_reader

import torch
from allennlp.data import Token, Vocabulary


from allennlp.data.fields import ListField, TextField

from allennlp.data.token_indexers import (
    TokenCharactersIndexer,SingleIdTokenIndexer
)


from allennlp.data.tokenizers import (
    CharacterTokenizer,
)

from allennlp.data.tokenizers import (
    SpacyTokenizer,
)

from allennlp.nn import util as nn_util

import phonemizer
from phonemizer import phonemize
from phonemizer import separator

import warnings
warnings.filterwarnings("ignore")

In [44]:
fo = open("encode.txt", "r")
alphabet=[]

for i in range(65):
  line = fo.readline()
  alphabet.append(line[0:-1])

# Close opend file
fo.close()


In [45]:
alphabet

['p',
 'ɹ',
 'ɪ',
 'n',
 't',
 'ŋ',
 'ð',
 'oʊ',
 'l',
 'i',
 's',
 'ɛ',
 'w',
 'tʃ',
 'iː',
 'ɑː',
 'æ',
 'z',
 'ə',
 'k',
 'ɜː',
 'd',
 'f',
 'ɚ',
 'ʌ',
 'm',
 'ɔː',
 'ɑːɹ',
 'ᵻ',
 'b',
 'ʃ',
 'v',
 'aɪ',
 'ʊ',
 'ɡ',
 'eɪ',
 'ɔːɹ',
 'oːɹ',
 'ɾ',
 'ɐ',
 'uː',
 'əl',
 'θ',
 'dʒ',
 'j',
 'aʊ',
 'h',
 'ɔ',
 'ɛɹ',
 'ʔ',
 'n̩',
 'ɪɹ',
 'ʊɹ',
 'aɪɚ',
 'ʒ',
 'oː',
 'iə',
 'r',
 'ɔɪ',
 'aɪə',
 'õ',
 '(gn)',
 'a',
 'ɣ',
 'x']

In [34]:
alphabet = list("abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}") + [" ", "  "]

In [53]:
vocab = Vocabulary()
vocab.add_tokens_to_namespace(
    alphabet,
    namespace='token_vocab')

#vocab.add_tokens_to_namespace(['DT', 'VBZ', 'NN', '.'], namespace='pos_tag_vocab')

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66]

In [64]:
tokenizer = SpacyTokenizer(pos_tags=True)
#tokenizer = CharacterTokenizer()
text = phonemize("Hello world!",backend='espeak',with_stress=False, separator=separator.Separator(phone=' ', syllable='', word=''))
text2 = phonemize("Single Id Token Indexer",backend='espeak',with_stress=False, separator=separator.Separator(phone=' ', syllable='', word=''))
#text, text2 = "hello world", "hello world hello world"
tokens = tokenizer.tokenize(text)
tokens2 = tokenizer.tokenize(text2)
print("Tokens:", tokens)
print("Tokens 2:", tokens2)

Tokens: [h, ə, l, oʊ, w, ɜː, l, d]
Tokens 2: [s, ɪ, ŋ, ɡ, əl, aɪ, d, iː, t, oʊ, k, ə, n, ɪ, n, d, ɛ, k, s, ɚ]


In [65]:
token_indexer = {
    'tokens': SingleIdTokenIndexer(namespace='token_vocab'),
}


In [66]:
text_field = TextField(tokens, token_indexer)
text_field.index(vocab)
text_field2 = TextField(tokens2, token_indexer)
text_field2.index(vocab)

In [73]:
print(text_field,"\n", text_field2)

TextField of length 8 with text: 
 		[h, ə, l, oʊ, w, ɜː, l, d]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 TextField of length 20 with text: 
 		[s, ɪ, ŋ, ɡ, əl, aɪ, d, iː, t, oʊ, k, ə, n, ɪ, n, d, ɛ, k, s, ɚ]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'}


In [71]:
padding_lengths = text_field2.get_padding_lengths()
tensor_dict = text_field.as_tensor(padding_lengths)
tensor_dict2 = text_field2.as_tensor(padding_lengths)
print("Combined tensor dictionary:", tensor_dict)
print("Combined tensor dictionary 2:", tensor_dict2)

Combined tensor dictionary: {'tokens': {'tokens': tensor([48, 20, 10,  9, 14, 22, 10, 23,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0])}}
Combined tensor dictionary 2: {'tokens': {'tokens': tensor([12,  4,  7, 36, 43, 34, 23, 16,  6,  9, 21, 20,  5,  4,  5, 23, 13, 21,
        12, 25])}}


In [ ]:
print(padding_lengths)

{'token_characters___token_characters': 35, 'token_characters___num_token_characters': 1}


In [ ]:
text_field_tensors = text_field.batch_tensors([tensor_dict, tensor_dict2])

In [ ]:
#print("Batched tensor dictionary:", text_field_tensors)

In [ ]:
x = text_field_tensors["token_characters"]["token_characters"]
x.shape

torch.Size([2, 35, 1])

In [ ]:
mask = nn_util.get_text_field_mask(text_field_tensors)
print("Mask:", mask)
print("Mask size:", mask.size())

Mask: tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True]])
Mask size: torch.Size([2, 35])


In [ ]:
list_field = ListField([text_field, text_field2])
tensor_dict = list_field.as_tensor(list_field.get_padding_lengths())
text_field_tensors = list_field.batch_tensors([tensor_dict])
print("Batched tensors for ListField[TextField]:", text_field_tensors)

In [ ]:
mask = nn_util.get_text_field_mask(text_field_tensors, num_wrapping_dims=1)
print("Mask:", mask)
print("Mask:", mask.size())

Mask: tensor([[[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False],
         [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True]]])
Mask: torch.Size([1, 2, 35])


In [ ]:
import allennlp.data.tokenizers.letters_digits_tokenizer as digit_token

In [ ]:
tokenizer = digit_token.LettersDigitsTokenizer()
sentence = "We are learning about TextFields"
tokens = tokenizer.tokenize(sentence)
tokens

[We, are, learning, about, TextFields]

In [ ]:
TSV = "train.tsv"

path = "/content"
tsvm = os.path.join(path, TSV)

with open(tsvm, "r") as tsv:
    walker = unicode_csv_reader(tsv, delimiter="\t")
    header = next(walker)
    print(header)
    print(len(header))
    walker = list(walker)
    print(len(walker))
    print(walker[0])

numpy_version = np.array(walker)
print(numpy_version.shape)

a, b = np.unique(numpy_version[:, 9], return_counts = True)
print(a.shape, b.shape)

c = np.concatenate((a[:, None], b[:, None]), axis=1)
print(c.shape)

['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment']
10
435947
['f1f6414c04e74453065e1b7fc1639c6f728dc03ed9589034b8531d8c7d8b994f223f2b79d5fcc42a2b7b19f8cbca5af08f31d47a554ddd682df04ba62caaaa56', 'common_voice_en_20009651.mp3', "It just didn't seem fair.", '2', '1', '', '', '', 'en', '']
(435947, 10)
(1,) (1,)
(1, 2)


In [ ]:
print(c)

[['' '435947']]


In [ ]:
a, b = np.unique(c[:, 1], return_counts = True)
hm = dict(zip(a, b))
print(hm)

{'1': 1801, '10': 735, '100': 6, '1000': 1, '101': 7, '1017': 1, '102': 5, '103': 11, '1035': 1, '104': 14, '105': 6, '106': 6, '107': 4, '108': 8, '109': 4, '1092': 1, '11': 234, '110': 5, '111': 7, '1113': 1, '1118': 1, '112': 4, '113': 2, '1136': 1, '114': 9, '115': 5, '1152': 1, '116': 6, '117': 4, '118': 4, '119': 14, '12': 249, '120': 3, '121': 6, '122': 6, '1220': 1, '123': 5, '124': 4, '125': 5, '126': 5, '127': 2, '1278': 1, '1279': 1, '128': 4, '129': 4, '13': 315, '130': 6, '131': 5, '132': 8, '133': 1, '134': 3, '135': 5, '136': 4, '1362': 1, '1375': 1, '138': 9, '14': 320, '140': 2, '141': 1, '1414': 1, '142': 7, '143': 4, '1435': 1, '144': 5, '146': 3, '147': 6, '1477': 1, '148': 4, '149': 6, '15': 292, '150': 4, '151': 5, '152': 1, '153': 3, '154': 3, '155': 2, '156': 6, '157': 5, '158': 2, '159': 2, '16': 166, '161': 2, '162': 1, '163': 8, '164': 3, '165': 2, '166': 3, '1669': 1, '167': 3, '168': 2, '169': 4, '17': 150, '170': 2, '171': 2, '1712': 1, '172': 6, '173': 1,

In [ ]:
c.shape

(15890, 2)

In [ ]:
sum = 0
for key, value in hm.items():
  sum+=int(value)

In [ ]:
sum

15890

In [ ]:
import collections
od = collections.OrderedDict(sorted(hm.items()))
#1801 + 716 + 406
od